In [7]:
from pyvis.network import Network
import numpy as np
import random as rand

In [9]:
class Graph:
    def __init__(self,
                 num_nodes: int = 0,
                 num_edges: int = 0) -> None:
        
        if not isinstance(num_nodes, int) or num_nodes < 0:
            raise ValueError("Input parameter 'num_nodes' must be a non-negative integer")
        if not isinstance(num_edges, int) or num_edges < 0:
            raise ValueError("Input parameter 'num_edges' must be a non-negative integer")

        self.network = Network()
        self.num_nodes = num_nodes
        self.num_edges = num_edges
        self.starting_node = None
        self.ending_node = None

    def generate_nodes(self) -> None:
        pass

    def generate_edges(self) -> None:
        pass

    def delete_edges(self) -> None:
        pass

    def shortest_path(self) -> None:
        pass

    def visualize_graph(self) -> None:
        pass
    